In [1]:
import numpy as np

with open('tasks/22.txt', 'r') as file:
    data = file.read().splitlines()

In [2]:
plan = data[-1]
mapa =  data[:-2]
max_len = max([len(x) for x in mapa])
mapa = [x + ' '*(max_len-len(x)) for x in mapa]
mapa = np.array([list(x) for x in mapa])

In [3]:
first_x = data[0].index('.')
first_y = 0

In [4]:
direction = 0 # 0-R, 1-D, 2-L, 3-U
dir_map ={
    'R': 1,
    'L': -1
}

move_map = {
    0: [0, 1],
    1: [1, 0],
    2: [0, -1],
    3: [-1, 0],
}

In [5]:
back_move_map = {
    (0, 1): 0,
    (1, 0): 1,
    (0, -1): 2,
    (-1, 0): 3,
}

In [6]:
def go(move, direction, position):
    for _ in range(move):
        y, x = position[0]+direction[0], position[1]+direction[1]
        if x == mapa.shape[1]:
            x = 0
        if x < 0:
            x = mapa.shape[1] - 1
        if y == mapa.shape[0]:
            y = 0
        if y < 0:
            y = mapa.shape[0] - 1
        next_symbol = mapa[y, x]
        if next_symbol == ' ':
            if direction == [0,1]:
                line = list(mapa[position[0]])
                first_dot = line.index('.') if '.' in line else 10000
                first_sharp = line.index('#') if '#' in line else 10000
                if first_dot < first_sharp:
                    position = [position[0], first_dot]
                else:
                    break

            elif direction == [1, 0]:
                line = list(mapa[:, position[1]])
                first_dot = line.index('.') if '.' in line else 10000
                first_sharp = line.index('#') if '#' in line else 10000
                if first_dot < first_sharp:
                    position = [first_dot, position[1]]
                else:
                    break
            
            elif direction == [0, -1]:
                line = list(mapa[position[0]][::-1])
                first_dot = line.index('.') if '.' in line else 10000
                first_sharp = line.index('#') if '#' in line else 10000
                if first_dot < first_sharp:
                    position = [position[0], mapa.shape[1] - 1 - first_dot]
                else:
                    break
            elif direction == [-1, 0]:
                line = list(mapa[:, position[1]][::-1])
                first_dot = line.index('.') if '.' in line else 10000
                first_sharp = line.index('#') if '#' in line else 10000
                if first_dot < first_sharp:
                    position = [mapa.shape[0] - 1 - first_dot, position[1]]
                else:
                    break        
                
        elif next_symbol == '#':
            break
        elif next_symbol == '.':
            position = y, x
        
            
    return position

In [7]:
y, x = first_y, first_x
direction = 0
move = ''
for i, p in enumerate(plan):
    if plan[i].isdigit():
        move += p
    else:
        y, x = go(int(move), move_map[direction], (y, x))
        move = ''
        direction = (direction+dir_map[p]) % 4
        continue
y, x = go(int(move), move_map[direction], (y, x))
x, y, = x+1, y+1

In [8]:
1000 * y + 4 * x + direction

117102

In [9]:
def go_2(move, direction, position):
    for _ in range(move):
        y, x = position[0]+direction[0], position[1]+direction[1]
        if x == mapa.shape[1] or x < 0 or y == mapa.shape[0] or y < 0:
            if x == mapa.shape[1]:
                x = 99 # border of 2nd face
                y = 149 - position[0]
                new_direction = [0, -1]
            elif x < 0:
                if y < 150: # 4th face
                    y = 149 - position[0]
                    x = 50
                    new_direction = [0, 1]
                elif 199 >= y >= 150:
                    x = position[0] - 100
                    y = 0
                    new_direction = [1, 0]
            elif y == mapa.shape[0]:
                x = position[1] + 100
                y = 0
                new_direction = direction 
            elif y < 0:
                if 100 > x > 49:
                    x = 0
                    y = position[1] + 100
                    new_direction = [0, 1]
                elif 149 >= x >= 100:
                    x = position[1] - 100
                    y = 199
                    new_direction = direction
        else:
            new_direction = direction
            next_symbol = mapa[y, x]
            if next_symbol == ' ':
                if direction == [0,1]:
                    if 50 <= y <= 99:
                        y = 49
                        x = position[0] + 50
                        new_direction = [-1, 0]
                    elif 100 <= y <= 149:
                        x = 149
                        y = 149 - position[0]
                        new_direction = [0, -1]
                    elif y > 149:
                        x = position[0] - 100
                        y = 149
                        new_direction = [-1, 0]
                elif direction == [1, 0]:
                    if x > 99:
                        x = 99
                        y = position[1] - 50
                        new_direction = [0, -1]
                    elif x <= 99:
                        x = 49
                        y = position[1] + 100
                        new_direction = [0, -1]
                elif direction == [0, -1]:
                    if y < 50:
                        x = 0
                        y = 149 - position[0]
                        new_direction = [0, 1]
                    elif y >= 50:
                        x = position[0] - 50
                        y = 100
                        new_direction = [1, 0]
                elif direction == [-1, 0]:
                    x = 50
                    y = position[1] + 50
                    new_direction = [0, 1]
                
        next_symbol = mapa[y, x]
        if next_symbol == '#':
            break
        elif next_symbol == '.':
            position = y, x
            direction = new_direction
        
            
    return position, direction

In [10]:
y, x = first_y, first_x
direction = 0
move = ''
for i, p in enumerate(plan):
    if plan[i].isdigit():
        move += p
    else:
        (y, x), new_direction = go_2(int(move), move_map[direction], (y, x)) 
        move = ''
        direction = (back_move_map[tuple(new_direction)]+dir_map[p]) % 4
        continue

(y, x), _ = go_2(int(move), move_map[direction], (y, x))
x, y, = x+1, y+1

In [11]:
1000 * y + 4 * x + direction

135297